## Preparation

### Check assigned GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 28 18:07:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Check assigned memory

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


### Download Object Detection API repo



In [1]:
!rm -rf object-detection-api && git clone https://github.com/CatchZeng/object-detection-api.git

Cloning into 'object-detection-api'...
remote: Enumerating objects: 370, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 370 (delta 22), reused 24 (delta 22), pack-reused 340
Receiving objects: 100% (370/370), 1.82 MiB | 37.27 MiB/s, done.
Resolving deltas: 100% (140/140), done.


### Install Object Detection API

In [2]:
!cd /content/object-detection-api && make install

if [ -d "./models" ]; then \
    echo 'models downloaded'; \
  else \
        git clone --depth=1 https://github.com/tensorflow/models; \
fi
Cloning into 'models'...
remote: Enumerating objects: 3598, done.
remote: Counting objects: 100% (3598/3598), done.
remote: Compressing objects: 100% (2996/2996), done.
remote: Total 3598 (delta 943), reused 1485 (delta 548), pack-reused 0
Receiving objects: 100% (3598/3598), 47.10 MiB | 15.29 MiB/s, done.
Resolving deltas: 100% (943/943), done.
cd models/research && \
protoc object_detection/protos/*.proto --python_out=. && \
cp object_detection/packages/tf2/setup.py . && \
python -m pip install --use-feature=2020-resolver .
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/object-detection-api/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-t

###Make Workspace


In [3]:
import os
os.chdir("/content/object-detection-api/")

!make workspace-box SAVE_DIR=workspace NAME=crosswalks

python scripts/workspace/box/workspace.py --save_dir=workspace --name=crosswalks
cp -r scripts/workspace/box/files/* workspace/crosswalks
cp models/research/object_detection/model_main_tf2.py workspace/crosswalks
cp models/research/object_detection/exporter_main_v2.py workspace/crosswalks
cp models/research/object_detection/export_tflite_graph_tf2.py workspace/crosswalks
touch workspace/crosswalks/annotations/label_map.pbtxt


### Get images

#### From Images

In [ ]:
import os
os.chdir("/content/object-detection-api/workspace/crosswalks")

from google.colab import drive
drive.mount('/content/drive/')

!cp -r /content/drive/MyDrive/CSCI1470/final-project/images /content/object-detection-api/workspace/crosswalks/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#### From Zip

In [ ]:
import os
os.chdir("/content/object-detection-api/workspace/crosswalks/images/")

# run this one
!curl -L "https://universe.roboflow.com/ds/zSQsp2GjZz?key=iqVpebPpKA" > roboflow.zip;
!unzip /content/object-detection-api/workspace/crosswalks/images/roboflow.zip -d /content/object-detection-api/workspace/crosswalks/images/
!mv /content/object-detection-api/workspace/crosswalks/images/valid /content/object-detection-api/workspace/crosswalks/images/val
!rm /content/object-detection-api/workspace/crosswalks/images/roboflow.zip /content/object-detection-api/workspace/crosswalks/images/README.dataset.txt /content/object-detection-api/workspace/crosswalks/images/README.roboflow.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   891  100   891    0     0    443      0  0:00:02  0:00:02 --:--:--   443
100 60.5M  100 60.5M    0     0  9489k      0  0:00:06  0:00:06 --:--:-- 14.0M
Archive:  /content/object-detection-api/workspace/crosswalks/images/roboflow.zip
 extracting: /content/object-detection-api/workspace/crosswalks/images/README.dataset.txt  
 extracting: /content/object-detection-api/workspace/crosswalks/images/README.roboflow.txt  
 extracting: /content/object-detection-api/workspace/crosswalks/images/test/Strisce--105-_jpg.rf.16ab0e51e3a52d6520b0c449d2a8b56b.jpg  
 extracting: /content/object-detection-api/workspace/crosswalks/images/test/Strisce--105-_jpg.rf.16ab0e51e3a52d6520b0c449d2a8b56b.xml  
 extracting: /content/object-detection-api/workspace/crosswalks/images/test/Strisce--105-_jpg.rf.e2fce3b37caa1e8452748ffbadde64df.jpg  
 extractin

#### From Zip and Drive

In [5]:
import os
os.chdir("/content/object-detection-api/workspace/crosswalks/images/")

from google.colab import drive
drive.mount('/content/drive/')

# run this one
!rm -r /content/object-detection-api/workspace/crosswalks/images
!mkdir /content/object-detection-api/workspace/crosswalks/images
!cp -r /content/drive/MyDrive/final-project/images/data.zip /content/object-detection-api/workspace/crosswalks/images/data.zip
!unzip /content/object-detection-api/workspace/crosswalks/images/data.zip -d /content/object-detection-api/workspace/crosswalks/images
!mv /content/object-detection-api/workspace/crosswalks/images/data/train /content/object-detection-api/workspace/crosswalks/images/train
!mv /content/object-detection-api/workspace/crosswalks/images/data/test /content/object-detection-api/workspace/crosswalks/images/test
!mv /content/object-detection-api/workspace/crosswalks/images/data/val /content/object-detection-api/workspace/crosswalks/images/val

!rm -r /content/object-detection-api/workspace/crosswalks/images/__MACOSX /content/object-detection-api/workspace/crosswalks/images/data.zip /content/object-detection-api/workspace/crosswalks/images/data

Streaming output truncated to the last 5000 lines.
  inflating: /content/object-detection-api/workspace/crosswalks/images/data/val/00_15_52_395565.xml  
  inflating: /content/object-detection-api/workspace/crosswalks/images/data/val/19_03_57_792925.jpg  
  inflating: /content/object-detection-api/workspace/crosswalks/images/data/val/18_08_23_763346.jpg  
  inflating: /content/object-detection-api/workspace/crosswalks/images/data/val/21_20_19_858094.jpg  
  inflating: /content/object-detection-api/workspace/crosswalks/images/data/val/13_40_13_656828.xml  
  inflating: /content/object-detection-api/workspace/crosswalks/images/data/val/03_57_42_484946.jpg  
  inflating: /content/object-detection-api/workspace/crosswalks/images/data/val/03_57_42_512930.jpg  
  inflating: /content/object-detection-api/workspace/crosswalks/images/data/val/03_57_42_511578.xml  
  inflating: /content/object-detection-api/workspace/crosswalks/images/data/val/16_13_43_724107.jpg  
  inflating: /content/object-de

### Make Record Files

In [6]:
import os
os.chdir("/content/object-detection-api/workspace/crosswalks")

!make gen-tfrecord

python gen_label_map.py
unsorted:  ['crosswalk']
sorted:  ['crosswalk']
item {
  id: 1
  name: 'crosswalk'
}

python generate_tfrecord.py \
-x images/train \
-l annotations/label_map.pbtxt \
-o annotations/train.record
Successfully created the TFRecord file: annotations/train.record
python generate_tfrecord.py \
-x images/val \
-l annotations/label_map.pbtxt \
-o annotations/val.record
Successfully created the TFRecord file: annotations/val.record


### Download pre-trained Mask R-CNN model

In [8]:
import os
os.chdir("/content/object-detection-api/workspace/crosswalks")

!model=faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz;
!curl -O http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz;
!tar zxvf /content/object-detection-api/workspace/crosswalks/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz;
!mv -f /content/object-detection-api/workspace/crosswalks/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8 ./pre-trained-models/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8
!rm -rf /content/object-detection-api/workspace/crosswalks/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8 /content/object-detection-api/workspace/crosswalks/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz;

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  337M  100  337M    0     0   120M      0  0:00:02  0:00:02 --:--:--  120M
faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/
faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint/
faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint/checkpoint
faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/pipeline.config
faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/saved_model/
faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/saved_model/saved_model.pb
faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/saved_model/variables/
faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/saved_model/variables/variabl

### Get Pipeline Config

In [11]:
import os
os.chdir("/content/object-detection-api/workspace/crosswalks")

from google.colab import drive
drive.mount('/content/drive/')

!mkdir /content/object-detection-api/workspace/crosswalks/models/faster_rcnn_resnet101_v1_640x640
!cp -r /content/drive/MyDrive/final-project/pipeline_rcnn_1.config /content/object-detection-api/workspace/crosswalks/models/faster_rcnn_resnet101_v1_640x640/pipeline.config

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Train model



In [ ]:
import os
os.chdir("/content/object-detection-api/workspace/crosswalks")

from google.colab import drive
drive.mount('/content/drive/')
!python model_main_tf2.py --model_dir=models/faster_rcnn_resnet101_v1_640x640 --pipeline_config_path=models/faster_rcnn_resnet101_v1_640x640/pipeline.config
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path models/faster_rcnn_resnet101_v1_640x640/pipeline.config --trained_checkpoint_dir models/faster_rcnn_resnet101_v1_640x640/ --output_directory exported-models/faster_rcnn_resnet101_v1_640x640;

!zip -r /content/exported-modles.zip /content/object-detection-api/workspace/crosswalks/exported-models
!cp -r /content/exported-modles.zip /content/drive/MyDrive/final-project/exported-modles1.zip
!zip -r /content/exported-modles-whole.zip /content/object-detection-api/workspace/crosswalks/models
!cp -r /content/exported-modles-whole.zip /content/drive/MyDrive/final-project/exported-modles-whole1.zip



Streaming output truncated to the last 5000 lines.
 'learning_rate': 0.0350368}
I1206 21:35:36.318245 140100414232448 model_lib_v2.py:708] {'Loss/BoxClassifierLoss/classification_loss': 0.020783875,
 'Loss/BoxClassifierLoss/localization_loss': 0.059537675,
 'Loss/RPNLoss/localization_loss': 0.006187533,
 'Loss/RPNLoss/objectness_loss': 0.0025189063,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 0.089027986,
 'learning_rate': 0.0350368}
INFO:tensorflow:Step 13100 per-step time 0.709s
I1206 21:36:47.170055 140100414232448 model_lib_v2.py:705] Step 13100 per-step time 0.709s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classification_loss': 0.0055174655,
 'Loss/BoxClassifierLoss/localization_loss': 0.017609196,
 'Loss/RPNLoss/localization_loss': 0.002200153,
 'Loss/RPNLoss/objectness_loss': 0.00024686038,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 0.025573675,
 'learning_rate': 0.034950163}
I1206 21:36:47.170337 140100414232448 model_lib_v2.py:708] {'Loss/BoxClassifierLoss/

### FAQ

```
ImportError: cannot import name '_registerMatType' from 'cv2.cv2'
```

> https://stackoverflow.com/questions/70537488/cannot-import-name-registermattype-from-cv2-cv2

Execute the following scripts.

In [ ]:
!pip list | grep opencv-python-headless
!pip uninstall --yes opencv-python-headless
!pip install opencv-python-headless==4.1.2.30

opencv-python-headless        4.5.5.64
Found existing installation: opencv-python-headless 4.5.5.64
Uninstalling opencv-python-headless-4.5.5.64:
  Successfully uninstalled opencv-python-headless-4.5.5.64
     |████████████████████████████████| 21.8 MB 1.4 MB/s 


```
2 root error(s) found.
  (0) UNIMPLEMENTED:  DNN library is not found.
     [[{{node functional_1/conv1_conv/Conv2D}}]]
     [[StatefulPartitionedCall/SecondStagePostprocessor/BatchMultiClassNonMaxSuppression/MultiClassNonMaxSuppression/Reshape_5/_126]]
  (1) UNIMPLEMENTED:  DNN library is not found.
     [[{{node functional_1/conv1_conv/Conv2D}}]]
0 successful operations.
```
> https://stackoverflow.com/questions/71000120/colab-0-unimplemented-dnn-library-is-not-found

Execute the following scripts.

In [ ]:
!pip install --upgrade tf-models-official==2.8.0
!pip install --upgrade tensorflow==2.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 29.5 MB/s 
     |████████████████████████████████| 497.9 MB 4.5 kB/s 
     |████████████████████████████████| 4.9 MB 49.3 MB/s 
     |████████████████████████████████| 5.8 MB 54.0 MB/s 
     |████████████████████████████████| 1.4 MB 56.5 MB/s 
     |████████████████████████████████| 462 kB 67.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.11.0
    Uninstalling tensorboard-2.11.0:
      Successfully uninstalled tensorboard-2.11.0
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
  Attempting un

## Export model

In [ ]:
!make export-640

make: *** No rule to make target 'export-640'.  Stop.


## Download models

In [ ]:
from google.colab import files

!zip -r /content/exported-modles.zip /content/object-detection-api/workspace/crosswalks/exported-models
files.download("/content/exported-modles.zip")

  adding: content/object-detection-api/workspace/crosswalks/exported-models/ (stored 0%)
  adding: content/object-detection-api/workspace/crosswalks/exported-models/ssd_mobilenet_v2_fpnlite_640x640_quant_lite/ (stored 0%)
  adding: content/object-detection-api/workspace/crosswalks/exported-models/ssd_mobilenet_v2_fpnlite_320x320_quant_lite/ (stored 0%)
  adding: content/object-detection-api/workspace/crosswalks/exported-models/ssd_mobilenet_v2_fpnlite_640x640/ (stored 0%)
  adding: content/object-detection-api/workspace/crosswalks/exported-models/ssd_mobilenet_v2_fpnlite_640x640/saved_model/ (stored 0%)
  adding: content/object-detection-api/workspace/crosswalks/exported-models/ssd_mobilenet_v2_fpnlite_640x640/saved_model/saved_model.pb (deflated 92%)
  adding: content/object-detection-api/workspace/crosswalks/exported-models/ssd_mobilenet_v2_fpnlite_640x640/saved_model/variables/ (stored 0%)
  adding: content/object-detection-api/workspace/crosswalks/exported-models/ssd_mobilenet_v2_f

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>